In [2]:
# !pip install mediapipe

In [50]:
import cv2
import numpy as np
import mediapipe as mp
import time

In [37]:
class WorkoutDetector():
    def __init__(self,  static_image_mode=False, model_complexity=1, smooth_landmarks=True,min_detection_confidence=0.5,
    min_tracking_confidence=0.5):
        
        self.static_image_mode = static_image_mode
        self.model_complexity = model_complexity
        self.smooth_landmarks = smooth_landmarks
        self.min_detection_confidence = min_detection_confidence 
        self.min_tracking_confidence = min_tracking_confidence
        
        self.mpDraw =  mp.solutions.drawing_utils
        self.mpPose =  mp.solutions.pose
        self.pose_landmark_map = mp.solutions.pose.PoseLandmark
        self.pose =  self.mpPose.Pose(
            self.static_image_mode,
            self.model_complexity,
            self.smooth_landmarks,
            self.min_detection_confidence,
            self.min_tracking_confidence,
        )
        
    
    
    def check_lm_visibility(self, landmarks, landmark_name):
        landmark_name = landmark_name.upper()
        visibility = landmarks[self.pose_landmark_map[landmark_name].value].visibility  * 100
        return visibility > 90
        
        
        
    def calculate_angle(self, image,land_marks, draw=True):
        pos_a, pos_b, pos_c = land_marks
        lm1, lm1_x , lm1_y = self.lmList[pos_a]
        lm2, lm2_x , lm2_y = self.lmList[pos_b]
        lm3, lm3_x , lm3_y = self.lmList[pos_c]
        radians = np.arctan2(lm3_y - lm2_y, lm3_x - lm2_x) - np.arctan2(lm1_y - lm2_y, lm1_x - lm2_x)
        angle = np.abs(radians * 180.0/np.pi)
        
        if angle > 180.0:
            angle = 360 - angle
        

#         print(angle)    
        if(draw):
#             cv2.putText(image, str(int(angle)), (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0,0), 3)
            cv2.line(image, (lm1_x, lm1_y), (lm2_x, lm2_y), (255,255,255),3)
            cv2.line(image, (lm2_x, lm2_y), (lm3_x, lm3_y), (255,255,255), 3)
            
            cv2.circle(image, (lm1_x, lm1_y), 15, (23,0,250), cv2.FILLED)
            cv2.circle(image, (lm2_x, lm2_y), 10, (10,0,250), cv2.FILLED)
            cv2.circle(image, (lm3_x, lm3_y), 5, (30,0,250), cv2.FILLED)
        return int(angle)

            
    
    
    
    
    def get_distance_btw_landmarks(self, image, lm_a, lm_b, draw=True):
        
        lm1, lm1_x , lm1_y = self.lmList[lm_a]
        lm2, lm2_x , lm2_y = self.lmList[lm_b]
        
        if draw:
            cv2.line(image, (lm1_x, lm1_y), (lm2_x, lm2_y), (255,255,255),3)
            cv2.circle(image, (lm1_x, lm1_y), 15, (23,0,250), cv2.FILLED)
            cv2.circle(image, (lm2_x, lm2_y), 15, (23,0,250), cv2.FILLED)
            
        return lm1_x - lm2_x
            
            
            
        
    def full_body_visible(self,landmarks, select_lms):
        
        nose_visibility = landmarks[self.mpPose.PoseLandmark.NOSE.value].visibility  * 100
        right_ankle_visibility = landmarks[self.mpPose.PoseLandmark.RIGHT_ANKLE.value].visibility  * 100
        left_ankle_visibility = landmarks[self.mpPose.PoseLandmark.LEFT_ANKLE.value].visibility  * 100
        
        return nose_visibility > 90 and right_ankle_visibility > 90 and left_ankle_visibility > 90

        
        
        
    def draw_angle(self, image, land_marks):
        pos_a, pos_b, pos_c = land_marks
        
        lm1, lm1_x , lm1_y = self.lmList[pos_a]
        lm2, lm2_x , lm2_y = self.lmList[pos_b]
        lm3, lm3_x , lm3_y = self.lmList[pos_c]
        
        cv2.line(image, (lm1_x, lm1_y), (lm2_x, lm2_y), (255,255,255),3)
        cv2.line(image, (lm2_x, lm2_y), (lm3_x, lm3_y), (255,255,255), 3)

        cv2.circle(image, (lm1_x, lm1_y), 15, (23,0,250), cv2.FILLED)
        cv2.circle(image, (lm2_x, lm2_y), 10, (10,0,250), cv2.FILLED)
        cv2.circle(image, (lm3_x, lm3_y), 5, (30,0,250), cv2.FILLED)
        
        
    def getPosition(self, img, draw=True, world=False):
        self.lmList = []
        
        curr_landmarks = self.results.pose_landmarks.landmark
        
        if self.results.pose_landmarks:
            for id, lm in enumerate(curr_landmarks):
                h, w, c = img.shape
#                 print(id, lm)
                # getting the pixel values
                cx, cy = int(lm.x * w), int(lm.y * h)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(image, (cx, cy), 5, (255,0,0), cv2.FILLED)
        return self.lmList

    
    def detect_landmarks(self,img):
        
        img =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # MAKE DETECTIONS
        self.results = self.pose.process(img)
        
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        if self.results.pose_landmarks:
            return self.results.pose_landmarks.landmark
            
        
        
    def findPose(self, img, draw=True):

        img =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # MAKE DETECTIONS
        self.results = self.pose.process(img)
        
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS )
        return img

                
            
        
        
        
        
        
        

In [25]:
# VIDEO FEED
detector = WorkoutDetector()

cap = cv2.VideoCapture(0)
while cap.isOpened():
    message = ''
    is_true = False
    ret, frame = cap.read()
    landmarks = detector.detect_landmarks(frame)
#     print(landmarks)
    if landmarks:
        is_true = detector.full_body_visible(landmarks)
    
        message = 'is true {}'.format(is_true)
    
    cv2.putText(frame, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,30), 3)
    
    cv2.imshow('Mediapipe Feed', frame)
    
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [5]:
landmarks[1]

TypeError: 'NormalizedLandmarkList' object is not subscriptable

In [7]:
detector = WorkoutDetector()

while True:
    image = cv2.imread(pushup_start)
    image = detector.findPose(image, draw=False)
    lmList = detector.getPosition(image, draw=False)
    
    if len(lmList) > 1:
        detector.calculate_angle(image,15,23,31)

    cv2.imshow('Image', image)
    
    if cv2.waitKey(10) & 0xff == ord('q'):
        break
cv2.destroyAllWindows()

# cv2.destroyAllWindows()

# Squat workout Pipeline

In [47]:
squatVid = 'squat/squatv4.mp4'
squatStartImg = 'squat/start.png'
squatMidImg = 'squat/mid.png'

In [6]:

detector = WorkoutDetector()

while True:
    image = cv2.imread(squatMidImg)
    image = detector.findPose(image, draw=False)
    lmList = detector.getPosition(image, draw=False)
    
    if len(lmList) > 1:
        detector.calculate_angle(image,23,25,27)

    cv2.imshow('Image', image)
    
    if cv2.waitKey(10) & 0xff == ord('q'):
        break
cv2.destroyAllWindows()

In [49]:
detector = WorkoutDetector()



def squat():
    cap = cv2.VideoCapture(squatVid)
    pTime = 0
    count = 0
    stage =None

    while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img, draw=False)

        if len(lmList) > 1:
            angle = int(detector.calculate_angle(image,[23,25,27], draw=False))
    #         detector.draw_angle(image,23,25,27)
            ankle_width = detector.get_distance_btw_landmarks(image,27, 28 )
            knee_width = detector.get_distance_btw_landmarks(image,25, 26 )



            mycount = "ankle: {}, knee: {} stage: {} ".format(int(ankle_width), knee_width, stage)

            if knee_width > ankle_width:
                if angle:
                    if angle >= 170  and stage == 'down':
                        stage = 'up'
                        count +=1
                    if angle <= 90:
                        stage = 'down'
        cv2.putText(image, mycount, (70,50), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0,0), 3)
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
squat()

error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-_8k9tw8n\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# Bicycle Crunches

## Bicycle crunche Stages
1. START_LEFT_LEG_BENTUP_RIGHT_LEG_STRAIGHT
2. START_RIGHT_LEG_BENTUP_LEFT_LEG_STRAIGHT
3. END_LEFT_STRAIGHT_RIGHT_LEG_BENTUP
4. END_RIGHT_STRAIGHT_LEFT_LEG_STRAIGHT






In [4]:
bicycle_crunche_path = 'bicycyle_crunches/'

end_left_leg_down = 'crunche_end_left_leg_down.png'
end_right_leg_down = 'crunche_end_right_leg_down.png'

start_left_leg_up = 'crunche_start_left_leg_up.png'
start_right_leg_up = 'crunche_start_right_leg_up.png'

# bicycle_crunche_path+start_left_leg_up

In [8]:
def bent(angle, angle_check):
    return angle < angle_check




detector = WorkoutDetector()

while True:
    image = cv2.imread(bicycle_crunche_path+start_right_leg_up)
    image = detector.findPose(image, draw=False)
    lmList = detector.getPosition(image, draw=False)


    cv2.imshow('Image', image)
    
    if cv2.waitKey(10) & 0xff == ord('q'):
        break
cv2.destroyAllWindows()

In [4]:
detector = WorkoutDetector()

cap = cv2.VideoCapture(bicycle_crunche_path+start_left_leg_up)

while True:
    success, img = cap.read()

    image = detector.findPose(img, draw=False)
    lmList = detector.getPosition(img)


    cv2.imshow('Image', image)

    if cv2.waitKey(10) & 0xff == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()



error: OpenCV(4.5.2) /tmp/pip-req-build-vrhoqk3o/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


## In out Jumps

In [54]:
jumps_path =  'in_out_jumps/'

In_and_Out_Jump_mp4 = 'In_and_Out_Jump.mp4'


in__out_jump_start = 'in__out_jump_start.png'


close_leg_squat_down = 'in_out_jump_close_leg_squat_down.png'

jump_legs_straight = 'in_out_jump_jump_legs_straight.png'

legs_wide_squat_start = 'in_out_jumps_legs_wide_squat_start.png'

leg_wide_squat_down = 'in_out_jump_leg_wide_squat_down.png'


In [55]:
leg_leg_lms = [23,25,27]
leg_right_lms = [24,26,28]

left_arm_lms = [11,13,15]
right_arm_lms = [12,14,16]


def bent(angle, angle_check=45):
    return angle < angle_check


def not_bent(angle, angle_check=150):
    return angle > angle_check and angle < 187

In [43]:
detector = WorkoutDetector()

distance_of_ankles_start = 0

while True:
    image = cv2.imread(jumps_path+jump_legs_straight)
    image = detector.findPose(image, draw=False)
    lmList = detector.getPosition(image)
    le_el = detector.calculate_angle(image,leg_leg_lms)
    ri_el = detector.calculate_angle(image,leg_right_lms)
    
    arm_left_angle = detector.calculate_angle(image,left_arm_lms)
    arm_right_angle = detector.calculate_angle(image,right_arm_lms)
    

    


    cv2.imshow('Image', image)
    
    if cv2.waitKey(10) & 0xff == ord('q'):
        break
cv2.destroyAllWindows()

In [56]:
detector = WorkoutDetector()

cap = cv2.VideoCapture(jumps_path+In_and_Out_Jump_mp4)

distance_of_ankles_start = 0
distance_of_ankles = 0
stage = None
count = 0

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img)
        
        
        landmarks = detector.detect_landmarks(img)
        detector.getPosition(img)    
        if landmarks:
            vis_nose = detector.check_lm_visibility(landmarks, 'nose')
            vis_right_ankle = detector.check_lm_visibility(landmarks, 'right_ankle')
            vis_left_ankle = detector.check_lm_visibility(landmarks, 'left_ankle')
            
            if vis_nose and vis_right_ankle or vis_left_ankle:
                left_leg = detector.calculate_angle(image,leg_leg_lms)
                right_leg = detector.calculate_angle(image,leg_right_lms)

                arm_left_angle = detector.calculate_angle(image,left_arm_lms)
                arm_right_angle = detector.calculate_angle(image,right_arm_lms)
                
                message = 'dis {} stage {} legs {} {} count {}'.format(distance_of_ankles, stage, left_leg, right_leg, count)
                color = (255,0, 0)
                if bent(arm_left_angle) and bent(arm_left_angle):
                    distance_of_ankles = detector.get_distance_btw_landmarks(image,27, 28)

                    if not_bent(left_leg) and not_bent(right_leg) and distance_of_ankles < 100:
                        if stage == 'STAGE_DOWN_TWO':
                            count += 1
                            stage = 'STAGE_START'
                        stage = 'STAGE_START'


                    if bent(left_leg, 150) and bent(right_leg, 150) and stage == 'STAGE_START':
                        stage = 'STAGE_DOWN_ONE'

                    if bent(left_leg, 150) and bent(right_leg, 150) and distance_of_ankles > 100:
                        stage = 'STAGE_DOWN_TWO'
#                 color = (3, 56, 11)
                
                
                
                    cv2.putText(image, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
        image = cv2.resize(image, (960, 540))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [23]:
cv2.putText??

## Donkey Kicks

In [57]:
path_donkey_kicks = 'donkey_kicks/'
Donkey_Kicks = 'Donkey_Kicks.mp4'

In [59]:
detector = WorkoutDetector()

cap = cv2.VideoCapture(path_donkey_kicks+Donkey_Kicks)

left_stage = None
right_stage = None

count = 0
left_count = 0
right_count = 0

color = (255,0, 0)

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        landmarks = detector.detect_landmarks(img)
        detector.getPosition(img)    
        if landmarks:
            vis_nose = detector.check_lm_visibility(landmarks, 'nose')
            vis_right_ankle = detector.check_lm_visibility(landmarks, 'right_ankle')
            vis_left_ankle = detector.check_lm_visibility(landmarks, 'left_ankle')
            
            if vis_nose and vis_right_ankle or vis_left_ankle:
                is_true = True

            left_leg = detector.calculate_angle(image,leg_leg_lms, draw=False)
            right_leg = detector.calculate_angle(image,leg_right_lms, draw=False)
            left_leg_rise = detector.calculate_angle(image,[11,23,25])
            right_leg_rise = detector.calculate_angle(image,[12,24,26])


            if bent(left_leg_rise, 125):
                left_stage = 'LEFT_LEG_DOWN_START'

            if not_bent(left_leg_rise, 160) and left_stage =='LEFT_LEG_DOWN_START':
                count +=1
                left_stage = 'LEFT_LEG_UP'

            if not_bent(right_leg_rise, 160) and right_stage =='RIGHT_LEG_DOWN_START':
                count +=1
                right_stage = 'RIGHT_LEG_UP'


            if bent(right_leg_rise, 125):
                right_stage = 'RIGHT_LEG_DOWN_START'

            msg = 'stage {}  leg_rise {}  count {} '.format(stage, left_leg_rise, count)


            cv2.putText(image, msg , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
        image = cv2.resize(image, (960, 540))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [67]:
detector = WorkoutDetector()



while True:
        image = cv2.imread(jumps_path+jump_legs_straight)
        image = detector.findPose(image, draw=False)
        
        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img)
        
        left_leg = detector.calculate_angle(image,leg_leg_lms, draw=False)
        right_leg = detector.calculate_angle(image,leg_right_lms, draw=False)
        left_leg_rise = detector.calculate_angle(image,[11,23,25])
        right_leg_rise = detector.calculate_angle(image,[12,24,26])
        
        message = 'legs {} {} leg_rise {} {} '.format(left_leg, right_leg, left_leg_rise, right_leg_rise)
        
        
        

                
                
        cv2.putText(image, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
        image = cv2.resize(image, (960, 540))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range

In [8]:
path_lunge = 'lunges/'
lunge_down = 'lunge_down.png'

lungeVid = 'Lunges.mp4'
lunges_start = 'lunges_start.png'

In [37]:
detector = WorkoutDetector()

cap = cv2.VideoCapture(path_lunge+lungeVid)

left_stage = None
right_stage = None

count = 0
left_count = 0
right_count = 0
dis = 0 

color = (255,0, 0)

while True:
        success, img = cap.read()

        image = detector.findPose(img, draw=False)
        lmList = detector.getPosition(img, draw=False)
        
        left_leg = detector.calculate_angle(image,leg_leg_lms, draw=True)
        right_leg = detector.calculate_angle(image,leg_right_lms, draw=True)
        
        arm_left_angle = detector.calculate_angle(image,left_arm_lms)
        arm_right_angle = detector.calculate_angle(image,right_arm_lms)
        dis = detector.get_distance_btw_landmarks(image,27, 28)
        
        message = 'dis {} angles {} > {} count {} stage {}'.format(dis, arm_left_angle, arm_right_angle, count, stage )
        
        
        
        if not_bent(left_leg, 140) and not_bent(right_leg, 140):
            if stage == 'RIGHT_LEG_DOWN_START':
                stage = 'START_STAGE'
                count += 1
                
            if stage == 'LEFT_LEG_DOWN_START':
                stage = 'START_STAGE'
                count += 1
        
        if bent(arm_left_angle, 90) and bent(arm_right_angle,90):
            if dis > 0 and dis < 10:
                stage = 'START_STAGE'
                
            if bent(left_leg, 100) and bent(right_leg, 100):

                if dis < 0:
                    stage = 'RIGHT_LEG_DOWN_START'
                elif dis > 150:
                    stage = 'LEFT_LEG_DOWN_START'

        
        
        cv2.putText(image, message , (70,50), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)

        image = cv2.resize(image, (960, 540))                # Resize image
        cv2.imshow('Image', image)

        if cv2.waitKey(10) & 0xff == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()




error: OpenCV(4.5.2) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-_8k9tw8n\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [26]:
-200  0

0